# LPSC - TP3

Testbench pour une seule itération de mandelbrot (``mandelbrot_iteration``)

In [1]:
from fixedpoint import FixedPoint
import numpy as np
import matplotlib.pyplot as plt
from mandelbrot_functions import mandelbrot_iter, screen, m, n, imag_values, real_values, MAX_ITER

In [3]:
# Nombre de testcases (à partir de zero)
Nzero = 10
# Nombre de testcases (avec un Z quelconque)
Nother = 100

TC = np.block([np.zeros(Nzero), np.ones(Nother)]) #0 -> Z=0, 1 -> Z=random

# Nom du fichier de testcases
# .csv -> permet de lire avec rainbow csv et avoir les couleurs
# .txt -> lecture par le VHDL
filename = "mandelbrot_iteration_testcases.csv" # le fichier .txt est également généré

# Fonction de raccourci pour fixed point
def FP(x):
    return FixedPoint(x, m=m, n=n, signed=True, str_base=2)

# File :
#
header = ["Cr", "Ci", "Zr_previous", "Zi_previous", "Zr_next", "Zi_next", "done_in", "R", "iterations_in", "iterations_out", "done_out"]
#
joiner = ','
R = FP(2)
file_data = ""

np.random.seed(0)
# print header
file_data += joiner.join(header) + '\n'
for i, tc in enumerate(TC):
    Cr = FP(np.random.choice(real_values))
    Ci = FP(np.random.choice(imag_values))
    if tc:
        Zr_previous = FP(np.random.choice(real_values))
        Zi_previous = FP(np.random.choice(imag_values))
    else:
        Zr_previous = FP(0)
        Zi_previous = FP(0)
    done_in = np.random.randint(0, 1+1)
    iterations_in = np.random.randint(0, MAX_ITER)

    # Do the iteration
    Zr_next, Zi_next, done_out, iterations_out = mandelbrot_iter(Cr, Ci, Zr_previous, Zi_previous, R, iterations_in, m=m, n=n, max_iter=MAX_ITER, done_in=done_in)

    values = (Cr, Ci, Zr_previous, Zi_previous, Zr_next, Zi_next, done_in, R, iterations_in, iterations_out, done_out)
    data = joiner.join([str(v) for v in values]) + '\n'
    file_data += data

    #print(f"L{i+2:3d} : ({Cr: .3f},{Ci: .3f} | {Zr_previous: .3f}, {Zi_previous: .3f}) -> ({Zr_next: .3f}, {Zi_next: .3f}) ({done_out})")


with open(filename, 'w') as f:
    f.write(file_data)
with open(filename.replace('.csv', '.txt'), 'w') as f:
    # Suppression de la première ligne et remplacement des virgules par des espaces
    f.write(file_data.split('\n', 1)[1].replace(joiner, ' '))

